## 4.4 Givens Rotations

In [ ]:
import numpy as np
from scripts.lu import backward

We implement the QR decomposition with Givens rotations. Here, the matrix $A$ is overwritten by $R$ and we obtain in addition the matrix $Q^T$.

In [ ]:
def qr_givens(A):
    n, m = A.shape
    QT = np.identity(n, dtype=A.dtype)
    
    for i in range(m):
        for j in range(i + 1, n):
            c, s = A[i, i], -A[j, i]
            nrm = np.sqrt(c**2 + s**2)
            c, s = c / nrm, s / nrm
            for k in range(i, m):
                t1, t2 = A[i, k], A[j, k]
                A[i, k] = c * t1 - s * t2
                A[j, k] = s * t1 + c * t2
            for k in range(n):
                t1, t2 = QT[i, k], QT[j, k]
                QT[i, k] = c * t1 - s * t2
                QT[j, k] = s * t1 + c * t2
    return QT

#### Example 4.20 (QR factorization using Givens rotations)

We apply the algorithm to the previously used system of equations.

In [ ]:
A = np.array([[1,    1,    1   ],
              [0.01, 0,    0.01],
              [0,    0.01, 0.01]], dtype=np.half)
A2 = A.astype(np.single)
b = np.array([1, 0, 0.02], dtype=np.half)
x_ex = np.array([-1, 1, 1])

QT = qr_givens(A)
print(QT)

In [ ]:
np.linalg.norm(QT @ QT.T - np.eye(3, dtype=np.single), 2)

In [ ]:
QTb = np.dot(QT, b)
x = backward(A, QTb)
print(x)

This is again very close to the exact solution $x=(-1, 1, 1)^T$. In particular, if we take into account that machine precision at `half` precision is about $\epsilon\approx 4 \times 10^{-4}$.

In [ ]:
err = np.linalg.norm(x - x_ex) / np.linalg.norm(x_ex)
print(f'||x - x_ex|| / ||x|| = {err:.4e}')

The relative 2-norm error is about the size of the machine precision (for `half` floating point numbers). Similarly, when we multiply $Q$ and $R$ together and test the orthogonality of $Q$, we have

In [ ]:
err = np.linalg.norm(A2 - QT.transpose() @ A, ord=2) / np.linalg.norm(A2, ord=2)
print(f'||A - QR||_2 / ||A||_2 = {err:.4e}')

In [ ]:
Id = np.identity(QT.shape[0], dtype=np.single)
err = np.linalg.norm(Id - QT @ QT.T, ord=2)
print(f'||I - Q*Q^T||_2 = {err:.4e}')

*Additional code details*
- `A2` and `Id` are stored in `single` precision so that we can compute the 2-norm.